In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Ge

In [2]:
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-15 23:52:07--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.65MB/s    in 0.2s    

2022-05-15 23:52:07 (5.65 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
amz_reviews_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
amz_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [7]:
print(amz_reviews_df.count())
amz_reviews_df =amz_reviews_df.dropna();
print(amz_reviews_df.count())

5331449
5330701


In [8]:
# Create the vine_table. DataFrame
# vine_df = df.select([""])
vine_df = amz_reviews_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [9]:
print(vine_df.count())

5330701


In [10]:
##Filter the new dataframe 
# WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.
# Load in a sql function to use columns
from pyspark.sql.functions import col
vine_over19votes_df = vine_df.filter(col("total_votes")  >= 20)

In [11]:
print(vine_over19votes_df.count())

133371


In [12]:
# Filter to new DF where the there were at least 50% helpful votes of total votes
vine_helpfulvotes_df = vine_over19votes_df.filter(col("helpful_votes")/col("total_votes")  >= 0.5)

In [ ]:
print(vine_helpfulvotes_df.count())

121322


In [13]:
# Filter further to new DF where review was part of Vine program (paid=='Y') 
vine_paid_df = vine_helpfulvotes_df.filter(col("vine")=='Y')

In [17]:
print(vine_paid_df.count())
total_paid = vine_paid_df.count()

497


In [18]:
# Filter further to new DF where review was NOT part of Vine program (paid=='N') 
vine_notpaid_df = vine_helpfulvotes_df.filter(col("vine")=='Y')
print(vine_notpaid_df.count())
total_not_paid = vine_notpaid_df.count()

120825


In [21]:
review_summary = vine_helpfulvotes_df.filter(col("star_rating")=="5").groupBy("vine").agg({"vine":"count"}).show()


+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|        220|
|   N|      74445|
+----+-----------+



In [24]:
## Positive reviews = 4 or 5 star
review_summary = vine_helpfulvotes_df.filter(col("star_rating").isin(["4","5"])).groupBy("vine").agg({"vine":"count"}).show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|        350|
|   N|      87462|
+----+-----------+



In [25]:
## Negative reviews = 1 or 2 star
review_summary = vine_helpfulvotes_df.filter(col("star_rating").isin(["1","2"])).groupBy("vine").agg({"vine":"count"}).show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|         70|
|   N|      26413|
+----+-----------+

